### 2-1 에러 핸들링

1. 값 대체

In [14]:
configuration = {"dbport": 5432}
configuration.get("dbport", "localhost")

5432

In [15]:
import os
os.getenv("DBHOST")

In [16]:
os.getenv("DPORT", 5432)

5432

### 2-2 예외처리 권장사항
1. 올바른 수준의 추상화 단계에서 예외 처리

- `deliver_evnet()` 
  - ConnectionError 과 ValueError의 관련성이 적다
  - ConnectionError는 `connect` 메서드에서 처리되어야 한다. 

In [17]:
import logging
import time

logger = logging.getLogger(__name__)

class DataTransport:
    """다른 레벨에서 예외를 처리하는 객체의 예"""

    retry_threshold: int = 5
    retry_n_times: int = 3

    def __init__(self, connector):
        self._connector = connector
        self.connection = None

    def deliver_event(self, event):
        try:
            self.connect()
            data = event.decode()
            self.send(data)
        except ConnectionError as e:
            logger.info("connection error detected: %s", e)
            raise
        except ValueError as e:
            logger.error("%r contains incorrect data: %s", event, e)
            raise

    def connect(self):
        for _ in range(self.retry_n_times):
            try:
                self.connection = self._connector.connect()
            except ConnectionError as e:
                logger.info(
                    "%s: attempting new connection in %is",
                    e,
                    self.retry_threshold,
                )
                time.sleep(self.retry_threshold)
            else:
                return self.connection
        raise ConnectionError(
            f"Couldn't connect after {self.retry_n_times} times"
        )

    def send(self, data):
        return self.connection.send(data)

- 수정
  - 다른 메서드나 함수로 분리
  - 연결 관리(ConnectionError)는 작은 함수로 나눔

In [18]:
def connect_with_retry(connector, retry_n_times, retry_threshold=5):
    """Tries to establish the connection of <connector> retrying
    <retry_n_times>.

    If it can connect, returns the connection object.
    If it's not possible after the retries, raises ConnectionError

    :param connector:           An object with a `.connect()` method.
    :param retry_n_times int:   The number of times to try to call
                                ``connector.connect()``.
    :param retry_threshold int: The time lapse between retry calls.

    """
    for _ in range(retry_n_times):
        try:
            return connector.connect()
        except ConnectionError as e:
            logger.info(
                "%s: attempting new connection in %is", e, retry_threshold
            )
            time.sleep(retry_threshold)
    exc = ConnectionError(f"Couldn't connect after {retry_n_times} times")
    logger.exception(exc)
    raise exc

class DataTransport:
    """An example of an object that separates the exception handling by
    abstraction levels.
    """

    retry_threshold: int = 5
    retry_n_times: int = 3

    def __init__(self, connector):
        self._connector = connector
        self.connection = None

    def deliver_event(self, event):
        self.connection = connect_with_retry(
            self._connector, self.retry_n_times, self.retry_threshold
        )
        self.send(event)

    def send(self, event):
        try:
            return self.connection.send(event.decode())
        except ValueError as e:
            logger.error("%r contains incorrect data: %s", event, e)
            raise

3. 원본 예외 포함
  - `raise <e> from <original_exception>` 구문 사용

In [19]:
class InternalDataError(Exception):
    """An exception with the data of our domain problem."""


def process(data_dictionary, record_id):
    try:
        return data_dictionary[record_id]
    except KeyError as e:
        raise InternalDataError("Record not present") from e

## 4. 개발 지침 약어

### 4-3. KIS

In [20]:
class ComplicatedNamespace:
    """An convoluted example of initializing an object with some properties.

    >>> cn = ComplicatedNamespace.init_with_data(
    ...    id_=42, user="root", location="127.0.0.1", extra="excluded"
    ... )
    >>> cn.id_, cn.user, cn.location
    (42, 'root', '127.0.0.1')

    >>> hasattr(cn, "extra")
    False

    """

    ACCEPTED_VALUES = ("id_", "user", "location")

    @classmethod
    def init_with_data(cls, **data):
        instance = cls()
        for key, value in data.items():
            if key in cls.ACCEPTED_VALUES:
                setattr(instance, key, value)
        return instance


In [21]:
cn = ComplicatedNamespace.init_with_data(
    id_=42, user="root", location="127.0.0.1", extra="excluded"
)
cn.id_, cn.user, cn.location

(42, 'root', '127.0.0.1')

- 초기화를 위해 불필요한 과정이 필요
- 파이썬에서 다른 객체를 초기화 할 때는 `__init__` 메서드 사용

In [22]:
#클린코드
class Namespace:
    """Create an object from keyword arguments."""


    ACCEPTED_VALUES = ("id_", "user", "location")

    def __init__(self, **data):
        accepted_data = {
            k: v for k, v in data.items() if k in self.ACCEPTED_VALUES
        }
        self.__dict__.update(accepted_data)

In [23]:
cn = Namespace(
     id_=42, user="root", location="127.0.0.1", extra="excluded"
)
cn.id_, cn.user, cn.location

(42, 'root', '127.0.0.1')

## 5. 컴포지션과 상속
> 상속안티패턴

- 안좋은 예
  - 원하는 기능은 수행하지만 비용 측면에서는 나쁨
  - 불필요한 수많은 메서드가 포함되어 있음

- 계층 구조가 잘못됨
  - class명만 가지고 사전타입인지 알 수 없음
- 결합력
  - 사전의 모든 메서드를 포함하지만 불필요 
  - 결합력이 낮다.

In [24]:
import collections
from datetime import datetime
from unittest import TestCase, main

class TransactionalPolicy(collections.UserDict):
    """Example of an incorrect use of inheritance."""

    def change_in_policy(self, customer_id, **new_policy_data):
        self[customer_id].update(**new_policy_data)

In [25]:
policy = TransactionalPolicy({
    "client001": {
        "fee": 1000.0,
        "expiration_date": datetime(2020, 1, 3),
    }
})
policy["client001"]


{'fee': 1000.0, 'expiration_date': datetime.datetime(2020, 1, 3, 0, 0)}

In [26]:
policy.change_in_policy("client001", expiration_date=datetime(2020, 1, 4))
policy["client001"]

{'fee': 1000.0, 'expiration_date': datetime.datetime(2020, 1, 4, 0, 0)}

In [27]:
dir(policy)

['_MutableMapping__marker',
 '__abstractmethods__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__copy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__ior__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__or__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__ror__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 'change_in_policy',
 'clear',
 'copy',
 'data',
 'fromkeys',
 'get',
 'items',
 'keys',
 'pop',
 'popitem',
 'setdefault',
 'update',
 'values']

- 리팩토링
  - 컴포지션을 사용
  - TransactionalPolicy 자체가 사전이 되는 것이 아니라 **사전을 활용**
  

In [28]:
from datetime import datetime
from unittest import TestCase, main


class TransactionalPolicy:
    """Example refactored to use composition."""

    def __init__(self, policy_data, **extra_data):
        self._data = {**policy_data, **extra_data}

    def change_in_policy(self, customer_id, **new_policy_data):
        self._data[customer_id].update(**new_policy_data)

    def __getitem__(self, customer_id):
        return self._data[customer_id]

    def __len__(self):
        return len(self._data)

### 5-3. 파이썬의 다중상속
- 파이썬은 C3 linearization 또는 MRO 알고리즘을 사용하여
충돌 문제를 해결

In [30]:
class BaseModule:
    module_name = "top"

    def __init__(self, module_name):
        self.name = module_name

    def __str__(self):
        return f"{self.module_name}:{self.name}"

class BaseModule1(BaseModule):
    module_name = "module-1"

class BaseModule2(BaseModule):
    module_name = "module-2"

class BaseModule3(BaseModule):
    module_name = "module-3"

class ConcreteModuleA12(BaseModule1, BaseModule2):
    """Extend 1 & 2

    >>> str(ConcreteModuleA12('name'))
    'module-1:name'
    """

class ConcreteModuleB23(BaseModule2, BaseModule3):
    """Extend 2 & 3

    >>> str(ConcreteModuleB23("test"))
    'module-2:test'
    """

In [31]:
str((ConcreteModuleA12("test")))

'module-1:test'

- 클래스의 결정 순서 출력

In [32]:
[cls.__name__ for cls in ConcreteModuleA12.mro()]

['ConcreteModuleA12', 'BaseModule1', 'BaseModule2', 'BaseModule', 'object']

> 믹스인

In [35]:
class BaseTokenizer:
    """
    >>> tk = BaseTokenizer("28a2320b-fd3f-4627-9792-a2b38e3c46b0")
    >>> list(tk)
    ['28a2320b', 'fd3f', '4627', '9792', 'a2b38e3c46b0']
    """

    def __init__(self, str_token):
        self.str_token = str_token

    def __iter__(self):
        yield from self.str_token.split("-")

In [36]:
tk = BaseTokenizer("28a2320b-fd3f-4627-9792-a2b38e3c46b0")
list(tk)

['28a2320b', 'fd3f', '4627', '9792', 'a2b38e3c46b0']

- mixin을 사용해 기본 클래스를 변경하지 않고 값을 대문자로 반환

In [38]:
class UpperIterableMixin:
    def __iter__(self):
        return map(str.upper, super().__iter__())


class Tokenizer(UpperIterableMixin, BaseTokenizer):
    """
    >>> tk = Tokenizer("28a2320b-fd3f-4627-9792-a2b38e3c46b0")
    >>> list(tk)
    ['28A2320B', 'FD3F', '4627', '9792', 'A2B38E3C46B0']
    """
    pass


In [39]:
tk = Tokenizer("28a2320b-fd3f-4627-9792-a2b38e3c46b0")
list(tk)

['28A2320B', 'FD3F', '4627', '9792', 'A2B38E3C46B0']

--- 

## 6. 함수와 메서드의 인자
> 인자는 함수에 어떻게 복사되는가

- list 와 같은 mutable 인자가 들어가면 값이 바뀔 수 있다.


In [41]:
def function(arg): 
    arg += " in function" 
    print(arg)

immutable = "hello" 
function(immutable)

mutable = list("hello") 
print(mutable)
function(mutable)
print(mutable) 

hello in function
['h', 'e', 'l', 'l', 'o']
['h', 'e', 'l', 'l', 'o', ' ', 'i', 'n', ' ', 'f', 'u', 'n', 'c', 't', 'i', 'o', 'n']
['h', 'e', 'l', 'l', 'o', ' ', 'i', 'n', ' ', 'f', 'u', 'n', 'c', 't', 'i', 'o', 'n']


> 가변인자

- 언패킹 훈련

In [44]:
def show(e, rest):
    print("요소: {0} - 나머지: {1}".format(e, rest))
    

In [45]:
first, *rest = [1, 2, 3, 4, 5]
show(first, rest)

요소: 1 - 나머지: [2, 3, 4, 5]



- `bad_users_from_rows` 레코드의 각 칼럼에 해당하는 값을 받아서 사용자를 생성
- `users_from_rows` 언패킹을 사용해 반복을 수행

In [43]:
USERS = [(i, f"first_name_{i}", f"last_name_{i}") for i in range(1_000)]


class User:
    def __init__(self, user_id, first_name, last_name):
        self.user_id = user_id
        self.first_name = first_name
        self.last_name = last_name

    def __repr__(self):
        return f"{self.__class__.__name__}({self.user_id!r}, {self.first_name!r}, {self.last_name!r})"


def bad_users_from_rows(dbrows) -> list:
    """A bad case (non-pythonic) of creating ``User``s from DB rows."""
    return [User(row[0], row[1], row[2]) for row in dbrows]


def users_from_rows(dbrows) -> list:
    """Create ``User``s from DB rows."""
    return [
        User(user_id, first_name, last_name)
        for (user_id, first_name, last_name) in dbrows
    ]


- 나쁜 코드인 이유
  - row[0], row[1] 이 무엇을 뜻하는지 알 수 없다.

## 7. SW 디자인 우수 사례 결론
### 7-1. 소프트웨어의 독립성
- 아래의 함수들은 독립적이다
- 다른 함수를 변경해도 나머지 함수가 그대로임

In [48]:
def calculate_price(base_price: float, tax: float, discount: float) -> float:
    """
    >>> calculate_price(10, 0.2, 0.5)
    6.0
    >>> calculate_price(10, 0.2, 0)
    12.0
    """
    return (base_price * (1 + tax)) * (1 - discount)

def show_price(price: float) -> str:
    """
    >>> show_price(1000)
    '$ 1,000.00'

    >>> show_price(1_250.75)
    '$ 1,250.75'
    """
    return "$ {0:,.2f}".format(price)

def str_final_price(
    base_price: float, tax: float, discount: float, fmt_function=str
) -> str:
    """
    >>> str_final_price(10, 0.2, 0.5)
    '6.0'
    >>> str_final_price(1000, 0.2, 0)
    '1200.0'
    >>> str_final_price(1000, 0.2, 0.1, fmt_function=show_price)
    '$ 1,080.00'

    """
    return fmt_function(calculate_price(base_price, tax, discount))

In [51]:
print(str_final_price(10, 0.2, 0.5))
print(str_final_price(1000, 0.2, 0))
print(str_final_price(1000, 0.2, 0.1, fmt_function=show_price))

6.0
1200.0
$ 1,080.00
